In [ ]:
# Name (שם מלא): Gal Davidi (גל דוידי)
# ID (תעודת זהות): 206555112

# Github link:
# https://github.com/gdavidi/Projects/tree/main/Yad2%20cars

In [25]:
import re
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz # pip install python-Levenshtein, pip install fuzzywuzzy
from datetime import datetime, timedelta

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error, r2_score,mean_squared_error,mean_absolute_error
from sklearn.model_selection import KFold
# import ppscore as pps # Might not work for you, need python verson 2.2 or lower.

import warnings
warnings.filterwarnings("ignore", category=UserWarning) # Suppress specific warning (OneHotEncoder gives this annoying warning that doesn't affect the code)

In [3]:
df = pd.read_csv("dataset.csv")

In [4]:
df

,manufactor,Year,model,Hand,Gear,capacity_Engine,Engine_type,Prev_ownership,Curr_ownership,Area,City,Price,Pic_num,Cre_date,Repub_date,Description,Color,Km,Test,Supply_score
0,יונדאי,2015,i35,2,אוטומטית,1600,בנזין,פרטית,פרטית,רעננה - כפר סבא,רעננה,51000.0,2.0,11/07/2023,11/07/2023,['רכב שמור בקנאות\nמוכרת עקב קבלת רכב חברה'],כחול כהה מטאלי,144000,NaN,NaN
1,ניסאן,2018,ניסאן מיקרה,1,אוטומטית,1200,בנזין,פרטית,פרטית,מושבים בשרון,אבן יהודה,49000.0,0.0,06/04/2022,22/05/2022,['שמורה כל התוספות'],כחול בהיר,69000,NaN,NaN
2,סוזוקי,2010,סוזוקי סוויפט,1,אוטומטית,1450,בנזין,NaN,NaN,רמת,רמת,22500.0,1.0,29/10/2022,29/10/2022,['רכב במצב מתוחזק ברמה גבוהה טסט עד אפריל 2023'],NaN,145000,NaN,NaN
3,טויוטה,2016,אוריס,1,טיפטרוניק,1600,בנזין,פרטית,פרטית,נס ציונה - רחובות,רחובות,63000.0,5.0,16/05/2024,16/05/2024,['אוטו במצב חדש!! שמור בקנאות!! נהג יחיד מטופל...,אפור מטאלי,27300,NaN,NaN
4,קיה,2012,פיקנטו,1,אוטומטית,1248,בנזין,NaN,NaN,"ראשל""צ והסביבה",ראשון לציון,37000.0,1.0,13/06/2022,13/06/2022,['שמור'],NaN,70000,NaN,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,סקודה,2016,סקודה אוקטביה (2016),2,אוטומטית,"1,400",בנזין,NaN,NaN,ראש העין והסביבה,כפר קאסם,60000.0,1.0,18/07/2022,18/07/2022,['רכב שמור &lt;br/&gt;&lt;br/&gt;פירוט:&lt;br/...,NaN,180,NaN,NaN
1496,אלפא רומיאו,2013,אלפא רומיאו ג'ולייטה,4,ידנית,1400,בנזין,NaN,NaN,חיפה וחוף הכרמל,חיפה,45000.0,0.0,44791,44791,['פרטית \r\nמכונית יפיפיה ללא שום תאונות ואו ת...,NaN,160000,NaN,NaN
1497,סקודה,2014,סקודה ראפיד (2014),1,אוטומטית,"1,400",בנזין,NaN,NaN,NaN,כפר מנדא,30000.0,1.0,30/08/2022,30/08/2022,['רכב נקי גיר שילדה מנוע במצב מעולה &lt;br/&gt...,NaN,NaN,NaN,NaN
1498,ניסאן,2011,ניסאן ג'וק JUKE,3,אוטומטית,1600,בנזין,פרטית,פרטית,אשדוד - אשקלון,אשדוד,28000.0,0.0,31/08/2023,09/11/2023,"['רכב מתוחזק היטב ללא תאונות או תקלות, טסט עד ...",אפור מטאלי,118000,81,NaN


### Functions I wrote and used throughout the assignment :

In [5]:
# Convert excel serial date to datetime object function
def excel_date_to_date(excel__serial_date):
	return datetime(1899, 12, 30) + timedelta(days=excel__serial_date)

# Extract the year and clean the text function
def extract_year_and_clean_text(text):
	# Regex pattern to find and remove the year (and surrounding characters)
	pattern = r'\s*\(?(\d{4})\)?,?\s*' 
	match = re.search(pattern, text)
	if match:
		year = match.group(1)  # Capture the year
		cleaned_text = re.sub(pattern, '', text)  # Remove the year and surrounding characters from the text
		return cleaned_text, year
	return text, None

# Cleaning the rows that contain years in the 'model' column by removing the year, and keeping only the model name
def clean_text_with_year(model_name):
	car_models = list(df['model'].unique())
	cleaned_model_name, year = extract_year_and_clean_text(model_name)
	if year and (model_name in car_models):
		return cleaned_model_name
	return model_name

# Finding the similarity between pairs of strings function
def find_similar_strings(strings, threshold=90):
	similar_pairs = []
	# Compare each string with every other string in the list
	for i in range(len(strings)):
		for j in range(i + 1, len(strings)):
			similarity = fuzz.ratio(strings[i], strings[j]) # Calculate similarity
			if similarity >= threshold:
				similar_pairs.append((strings[i], strings[j], similarity))
	return similar_pairs

# Print similar string values in a series function
def print_similar_string_values_in_series(series, threshold=90):
	strings = list(series.unique())
	# Finding similar strings
	similar_strings = find_similar_strings(strings, threshold)
	for pair in similar_strings:
		print(f"Pair: {pair[0]}\nand {pair[1]}\nwith similarity {pair[2]}%\n\n")
		
# Description cleaning function
def clean_description(desc):
	# Remove punctuation and non-alphabetic(english) characters
	desc = re.sub(r"[^א-ת\s]", "", desc)
	# Tokenize the text
	tokens = desc.split()
	return tokens

# Identify common meaning words function
def select_common_word(tokens):
	# Common meaning words (that logically have correlation with price)
	common_meaning_words = {'מטופל', 'חדש', 'שמור', 'מתוחזק', 'תוספות', 'מצוין', 'מעולה', 'נהדר','שמורה','ללא','מיבוא','חסכוני','מצויין','טיפול'}
	for word in tokens:
		if word in common_meaning_words:
			return word
	return 'אחר'  # If no common word is found, return 'אחר'       
		
# Cleaning the 'Description' column, selecting common meaning words, and replacing some words with synonyms (/similar words)
def description_features(df):
	df['Description'] = df['Description'].apply(clean_description)
	df['Description'] = df['Description'].apply(select_common_word)
	df['Description'] = df['Description'].replace('שמורה','שמור')
	df['Description'] = df['Description'].replace('מעולה','מצוין')
	df['Description'] = df['Description'].replace('נהדר','מצוין')
	df['Description'] = df['Description'].replace('טיפול','מטופל')
	df['Description'] = df['Description'].replace('מתוחזק','מטופל')
	df['Description'] = df['Description'].replace('מצויין','מצוין')
	  
# Feature engineering/handling for dates
def to_date(df, threshold=0.95):
	try:
		date_regex_pattern = r'^\d{2}/\d{2}/\d{4}$' # Date format: dd/mm/yyyy using regex pattern
		non_matching_rows_Cre_date = df[~(df['Cre_date'].str.contains(date_regex_pattern, regex=True))]
		non_matching_rows_Repub_date = df[~(df['Repub_date'].str.contains(date_regex_pattern, regex=True))]
		df.loc[non_matching_rows_Cre_date.index, 'Cre_date'] = df.loc[non_matching_rows_Cre_date.index, 'Cre_date'].astype('Int64').apply(excel_date_to_date).dt.strftime('%d/%m/%Y')
		df.loc[non_matching_rows_Repub_date.index, 'Repub_date'] = df.loc[non_matching_rows_Repub_date.index, 'Repub_date'].astype('Int64').apply(excel_date_to_date).dt.strftime('%d/%m/%Y')
		df['Cre_date'] = pd.to_datetime(df['Cre_date'], format="%d/%m/%Y") # Converting to datetime format
		df['Repub_date'] = pd.to_datetime(df['Repub_date'], format="%d/%m/%Y")
		# Check correlation between the two columns using each date's ordinal value 
		dates_features_df = df.copy()[['Price']]
		dates_features_df['Cre_date_ordinal'] = df['Cre_date'].apply(lambda date: date.toordinal())
		dates_features_df['Repub_date_ordinal'] = df['Repub_date'].apply(lambda date: date.toordinal())
		corr_matrix = dates_features_df[['Cre_date_ordinal', 'Repub_date_ordinal']].corr()
		if min(corr_matrix['Cre_date_ordinal']) >= threshold: # correlation score threshold
			df = df.drop(['Cre_date'], axis=1)
			dates_features_df = dates_features_df.drop(columns=['Cre_date_ordinal'])
		# --- day of the week ---
		dates_features_df['Repub_date_dayofweek'] = df['Repub_date'].dt.dayofweek
		# --- day of the month ---
		dates_features_df['Repub_date_day'] = df['Repub_date'].dt.day
		# --- month ---
		dates_features_df['Repub_date_month'] = df['Repub_date'].dt.month
		# --- year ---
		dates_features_df['Repub_date_year'] = df['Repub_date'].dt.year
		# Check PPS score between the new date features and the Price 
		# pps_matrix = pps.matrix(dates_features_df)[['x', 'y', 'ppscore']].pivot(columns='x', index='y', values='ppscore') #          <--- (Remove # before pps_matrix if you have ppscore installed)
		# --- Plot the heatmap --- (Remove # to plot the heatmap)
		# plt.figure(figsize=(12, 10))
		# sns.heatmap(pps_matrix, annot=True, cmap='viridis', cbar=True, linewidths=0.5)
		# plt.title('Predictive Power Score (PPS) Heatmap for All Features')
		# plt.xlabel('')
		# plt.ylabel('')
		# plt.yticks(rotation=0)
		# plt.show()
		# We can see that the new date features have a low predictive power score, therefore I will remove both columns Cre_date and Repub_date from the original dataframe
	except Exception:
		print("An error occured while handling the dates.")
		
# Get the season of a date function
def get_season(date):
	year = date.year
	seasons = {
		'winter': pd.date_range(start=f'01/12/{year}', end=f'31/12/{year}'),
		'winter': pd.date_range(start=f'01/01/{year}', end=f'28/02/{year}'),
		'spring': pd.date_range(start=f'01/03/{year}', end=f'31/05/{year}'),
		'summer': pd.date_range(start=f'01/06/{year}', end=f'31/08/{year}'),
		'autumn': pd.date_range(start=f'01/09/{year}', end=f'30/11/{year}'),
	}
	for season in seasons:
		if date in seasons[season]:
			return season
	return None      
		
# Cleaning the Prev_ownership column, replacing לא מוגדר, אחר with פרטית because it is pretty safe and logical to assume that only private cars are being sold on 'yad2' websites
# As well as replacing השכרה, חברה, ממשלתי with ליסינג because they all have basically the same meaning, regarding the context of the column and the prediction column
def ownership_features(df):
	df['Prev_ownership'] = df['Prev_ownership'].replace('השכרה', 'ליסינג')
	df['Prev_ownership'] = df['Prev_ownership'].replace('חברה', 'ליסינג')
	df['Prev_ownership'] = df['Prev_ownership'].replace('ממשלתי', 'ליסינג')
	df['Prev_ownership'] = df['Prev_ownership'].replace('לא מוגדר', 'פרטית')
	df['Prev_ownership'] = df['Prev_ownership'].replace('אחר', 'פרטית')
	df['Prev_ownership'] = df['Prev_ownership'].fillna(df['Prev_ownership'].mode()[0])

In [6]:
df['model'].unique()

array(['i35', 'ניסאן מיקרה', 'סוזוקי סוויפט', 'אוריס', 'פיקנטו',
       'אאודי A1', 'אימפרזה', 'ASX', '220', '525', 'מוקה', 'פורטה', ' Q3',
       'סיוויק סדאן', 'סוזוקי SX4 קרוסאובר', 'קורולה', 'גולף', 'פאסאט',
       'ספארק', 'מאזדה 3', 'ניסאן נוט', 'סול', 'V40 CC', 'לנסר ספורטבק',
       'i10', 'אאודי A3', ' A1', 'סקודה פאביה\r\n (2012)', 'אוקטביה',
       'CIVIC', 'איוניק', 'סונטה', 'i30', 'C-HR', 'מאליבו', 'ריו',
       'פוקוס', 'סקודה אוקטביה (2014)', 'X1', 'אוואו', 'סיוויק',
       'סקודה ראפיד (2015)', ' E-Class', ' S7', 'אפלנדר', 'SVX',
       'סוזוקי איגניס', 'ספייס סטאר', 'לקסוס IS300h', "גראנד, וויאג'ר",
       'C4', '2008', 'סטוניק', 'פולו', 'S60', 'אאודי RS5', "ג'אז הייבריד",
       'סוזוקי SX4', 'ג`טה', ' A4', 'אס-מקס', 'נירו', 'אינסייט',
       'רנו קליאו', '3', 'אאודי All Road', 'פאסאט CC', ' S-Class',
       'CADDY COMBI', 'אסטרה', 'XV', 'סיוויק סדאן החדשה', 'אאודי A5',
       '316', 'C3', 'סדרה 5', 'אקורד', 'i25', 'C1', 'יאריס',
       'לקסוס IS250', 'V40', 'סדרה 1',

In [7]:
df['model'].unique()

array(['i35', 'ניסאן מיקרה', 'סוזוקי סוויפט', 'אוריס', 'פיקנטו',
       'אאודי A1', 'אימפרזה', 'ASX', '220', '525', 'מוקה', 'פורטה', ' Q3',
       'סיוויק סדאן', 'סוזוקי SX4 קרוסאובר', 'קורולה', 'גולף', 'פאסאט',
       'ספארק', 'מאזדה 3', 'ניסאן נוט', 'סול', 'V40 CC', 'לנסר ספורטבק',
       'i10', 'אאודי A3', ' A1', 'סקודה פאביה\r\n (2012)', 'אוקטביה',
       'CIVIC', 'איוניק', 'סונטה', 'i30', 'C-HR', 'מאליבו', 'ריו',
       'פוקוס', 'סקודה אוקטביה (2014)', 'X1', 'אוואו', 'סיוויק',
       'סקודה ראפיד (2015)', ' E-Class', ' S7', 'אפלנדר', 'SVX',
       'סוזוקי איגניס', 'ספייס סטאר', 'לקסוס IS300h', "גראנד, וויאג'ר",
       'C4', '2008', 'סטוניק', 'פולו', 'S60', 'אאודי RS5', "ג'אז הייבריד",
       'סוזוקי SX4', 'ג`טה', ' A4', 'אס-מקס', 'נירו', 'אינסייט',
       'רנו קליאו', '3', 'אאודי All Road', 'פאסאט CC', ' S-Class',
       'CADDY COMBI', 'אסטרה', 'XV', 'סיוויק סדאן החדשה', 'אאודי A5',
       '316', 'C3', 'סדרה 5', 'אקורד', 'i25', 'C1', 'יאריס',
       'לקסוס IS250', 'V40', 'סדרה 1',

In [8]:
# Checking if there are any duplicates
print(df.duplicated().sum())
# There are 128 duplicates in the dataset, therefore I will remove them in the main preprocess function (while keeping only the first apperence)

128


In [9]:
# Check data types of each column
df.dtypes

manufactor          object
Year                 int64
model               object
Hand                 int64
Gear                object
capacity_Engine     object
Engine_type         object
Prev_ownership      object
Curr_ownership      object
Area                object
City                object
Price              float64
Pic_num            float64
Cre_date            object
Repub_date          object
Description         object
Color               object
Km                  object
Test                object
Supply_score       float64
dtype: object

In [22]:
# Check unique values of each column
for col in df.columns:
	print(f'{col} {df[col].unique()}\n')

manufactor ['יונדאי' 'ניסאן' 'סוזוקי' 'טויוטה' 'קיה' 'אאודי' 'סובארו' 'מיצובישי'
 'מרצדס' 'ב.מ.וו' 'אופל' 'הונדה' 'פולקסווגן' 'שברולט' 'מאזדה' 'וולוו'
 'סקודה' 'פורד' 'Lexsus' 'קרייזלר' 'סיטרואן' "פיג'ו" 'רנו' 'לקסוס'
 'דייהטסו' 'מיני' 'אלפא רומיאו']

Year [2015 2018 2010 2016 2012 2009 2003 2017 2013 2008 2014 2007 2011 2020
 2023 1988 2021 2019 1990 2004 1999 2005 2022 2006 2002 1983 1998 2000
 1995]

model ['i35' 'ניסאן מיקרה' 'סוזוקי סוויפט' 'אוריס' 'פיקנטו' 'אאודי A1' 'אימפרזה'
 'ASX' '220' '525' 'מוקה' 'פורטה' ' Q3' 'סיוויק סדאן'
 'סוזוקי SX4 קרוסאובר' 'קורולה' 'גולף' 'פאסאט' 'ספארק' 'מאזדה 3'
 'ניסאן נוט' 'סול' 'V40 CC' 'לנסר ספורטבק' 'i10' 'אאודי A3' ' A1'
 'סקודה פאביה\r\n (2012)' 'אוקטביה' 'CIVIC' 'איוניק' 'סונטה' 'i30' 'C-HR'
 'מאליבו' 'ריו' 'פוקוס' 'סקודה אוקטביה (2014)' 'X1' 'אוואו' 'סיוויק'
 'סקודה ראפיד (2015)' ' E-Class' ' S7' 'אפלנדר' 'SVX' 'סוזוקי איגניס'
 'ספייס סטאר' 'לקסוס IS300h' "גראנד, וויאג'ר" 'C4' '2008' 'סטוניק' 'פולו'
 'S60' 'אאודי RS5' "ג'אז הייבריד" 'סוזוק

In [23]:
car_models = list(df['model'].unique())
print(pd.Series(car_models).values)

['i35' 'ניסאן מיקרה' 'סוזוקי סוויפט' 'אוריס' 'פיקנטו' 'אאודי A1' 'אימפרזה'
 'ASX' '220' '525' 'מוקה' 'פורטה' ' Q3' 'סיוויק סדאן'
 'סוזוקי SX4 קרוסאובר' 'קורולה' 'גולף' 'פאסאט' 'ספארק' 'מאזדה 3'
 'ניסאן נוט' 'סול' 'V40 CC' 'לנסר ספורטבק' 'i10' 'אאודי A3' ' A1'
 'סקודה פאביה\r\n (2012)' 'אוקטביה' 'CIVIC' 'איוניק' 'סונטה' 'i30' 'C-HR'
 'מאליבו' 'ריו' 'פוקוס' 'סקודה אוקטביה (2014)' 'X1' 'אוואו' 'סיוויק'
 'סקודה ראפיד (2015)' ' E-Class' ' S7' 'אפלנדר' 'SVX' 'סוזוקי איגניס'
 'ספייס סטאר' 'לקסוס IS300h' "גראנד, וויאג'ר" 'C4' '2008' 'סטוניק' 'פולו'
 'S60' 'אאודי RS5' "ג'אז הייבריד" 'סוזוקי SX4' 'ג`טה' ' A4' 'אס-מקס'
 'נירו' 'אינסייט' 'רנו קליאו' '3' 'אאודי All Road' 'פאסאט CC' ' S-Class'
 'CADDY COMBI' 'אסטרה' 'XV' 'סיוויק סדאן החדשה' 'אאודי A5' '316' 'C3'
 'סדרה 5' 'אקורד' 'i25' 'C1' 'יאריס' 'לקסוס IS250' 'V40' 'סדרה 1'
 'סקודה אוקטביה (2013)' 'סראטו' 'מאזדה 5' 'סוזוקי סוויפט החדשה' 'מאזדה 2'
 '5' 'רנו קליאו דור 4' 'קורבט' "אטראז'" 'i20' ' A3' '200' 'B4' "ג'טה"
 '308' "סיוויק האצ'בק החדשה" 'ס

In [24]:
models_with_year = []
years = []
# Extacting rows that their value in column 'model' contains a year 
for model in car_models:
	cleaned_text, year = extract_year_and_clean_text(model)
	if year:
		models_with_year.append(model)
		years.append(year)
	continue
print(pd.Series(years).values) # Checking if my code works correctly
# There is an outlier in the years, '5008' isn't a year, it's probably a model name that had the same format as the years, so my regax pattern extracted it as well.
# I will go back to the code above where I see the output of pd.Series(car_models).values and try to recognize more outliers
# GT3000, 5008, 2008 are all outliers(2008 is 'Peugeot 2008', etc...), therefore I will remove them further down in the code (in the main preprocess function)
len(years) # All this work to fix just 37 rows :-|  (37 minus 3 outliers = 34 rows to be precise)

# Notice: I will clean the rows that contain years in the 'model' in the main preprocess function

['2012' '2014' '2015' '2008' '2013' '2015' '2016' '2013' '2013' '2016'
 '2015' '2015' '2016' '2015' '2014' '2011' '2016' '2015' '2012' '2010'
 '2012' '2014' '2014' '2012' '2014' '2014' '3000' '2011' '2013' '2013'
 '2016' '5008' '2012' '2015' '2012' '2015' '2011']


37

In [13]:
# Checking if Prev_ownership and Curr_ownership have similar values, so I could remove one of them
curr_prev_similir_num = len(df.loc[df['Prev_ownership'] == df['Curr_ownership']]) # 590 rows have the same value in both columns (not including missing values!)
prev_nan_num = df[['Prev_ownership']].isnull().sum().iloc[0] # 729 rows have missing values in Prev_ownership 
curr_nan_num = df[['Curr_ownership']].isnull().sum().iloc[0] # 728 rows have missing values in Curr_ownership
print(f'''There are {curr_prev_similir_num + curr_nan_num} rows that have the same value in both columns (including missing values)
out of {len(df)}, thats {(curr_prev_similir_num + curr_nan_num)/len(df):.2f}% !
Therefore, I will be removing the Curr_ownership column (in the main preprocess function)''') 

There are 1318 rows that have the same value in both columns (including missing values)
out of 1500, thats 0.88% !
Therefore, I will be removing the Curr_ownership column (in the main preprocess function)


# Pre Proccesing Function:

In [18]:
df = pd.read_csv("dataset.csv") 
# NOTICE: !* Run this cell again everytime you want to re-run prepare_data() function, running it multiple consecutive times without re-reading the dataset will cause errors *!

In [19]:
def prepare_data(df):
    
	# Remove duplicates
	df = df.drop_duplicates(keep='first').reset_index(drop=True)

	# Remove white spaces from all string columns
	string_columns = df.select_dtypes(include=['object']).columns
	for col in string_columns: 
		df[col] = df[col].str.strip().str.replace(r'[\r\n\t]', '', regex=True) 

	# Remove columns that...
	df = df.drop(['Test', 'Supply_score'], axis=1) # Removed due to having too many missing values
	df = df.drop(['Color', 'Area', 'City', 'Pic_num'], axis=1) # Removed due to not having any 'logical' meaning correlation with the price
	df = df.drop(['Curr_ownership'], axis=1) # Removed because it shares 0.88% of the same values with Curr_ownership
	df = df.dropna(how='any', subset=['Year', 'model', 'manufactor', 'Gear']) # Removed rows with missing values in these columns because they are important for the model

	df['manufactor'] = df['manufactor'].replace('Lexsus', 'לקסוס') 
	df['Gear'] = df['Gear'].replace('אוטומטי', 'אוטומטית')

	df['capacity_Engine'] = df['capacity_Engine'].str.replace(',','').astype('Int64')
	df['capacity_Engine'] = df['capacity_Engine'].fillna(df['capacity_Engine'].median().astype('int64')).astype('int64')
		
	df['Km'] = df['Km'].str.replace(',','').astype('Int64')
	df['Km'] = df['Km'].fillna(df['Km'].median().astype('int64')).astype('int64')
	df = df[df['Km'] < 400000].reset_index(drop=True) # Removing outliers (Car that drove more than 400,000 km shouldn't drive again! All the more so sell it)

	df['Engine_type'] = df['Engine_type'].replace('היבריד','היברידי')
	df['Engine_type'] = df['Engine_type'].replace('לא מוגדר', np.nan)
	df['Engine_type'] = df['Engine_type'].fillna(df['Engine_type'].mode()[0])

	to_date(df) # Cleaning invalid dates and changing all values type to datetime 
	ownership_features(df) # Explaination in the function itself (at the beggining of the code)
	description_features(df) # Explaination in the function itself (at the beggining of the code)

	# Creating new column named 'Season' that contains the season of each date in 'Repub_date' column, thinking it might hold some correlation with the price. In any case, it's better than having useless dates
	df['Season'] = df['Repub_date'].apply(get_season)
	df['Season'] = df['Season'].fillna(df['Season'].mode()[0])

	df = df.drop(['Cre_date', 'Repub_date'], axis=1) # Removed due to having low predictive power score with Price, and creatingg Season column instead (feature engineering)

	# Cleaning the rows that contain years in the 'model' column by removing the year, and keeping only the model name
	car_models = list(df['model'].unique())
	car_models.remove('GT3000')
	car_models.remove('5008')
	car_models.remove('2008')
	df['model'] = df['model'].apply(lambda model_name: extract_year_and_clean_text(model_name)[0] if extract_year_and_clean_text(model_name)[1] and (model_name in car_models) else model_name)
 
 
 # ---------------------------------------------------------------------------------------------- Model Buiilding Section ----------------------------------------------------------------------------------------------
 
	categorical_features = ['manufactor', 'model', 'Gear', 'Engine_type', 'Prev_ownership','Description', 'Season']
	numerical_features = ['Year', 'capacity_Engine', 'Hand', 'Km']
	all_x_columns = [col for col in df.columns if col in categorical_features or col in numerical_features]

	# Split the data into train and test sets
	X = df[all_x_columns]
	y = df['Price']

	# Preprocessing for numerical and categorical data
	numerical_transformer = StandardScaler()
	categorical_transformer = OneHotEncoder(handle_unknown='ignore', drop='first') # dropping the first column for each feature to avoid multicollinearity

	# Combined preprocessing for numerical and categorical features
	preprocessor = ColumnTransformer(
		transformers=[
			('num', numerical_transformer, numerical_features),
			('cat', categorical_transformer, categorical_features)
		])

	# Defining the model
	model = ElasticNet(random_state=42)

	# Creating a pipeline with the preprocessor and the model
	pipeline = Pipeline(steps=[('preprocessor', preprocessor),
							('model', model)])

	# Defining hyperparameters to tune, as part of finding the most suitable ones
	param_grid = {'model__alpha': [0.1, 1.0, 10.0]
				,'model__l1_ratio': [0.1, 0.5, 1]}

	cv = KFold(n_splits=10, shuffle=True, random_state=42)

	# Use GridSearchCV to search for the best hyperparameters
	model_wbest_params = GridSearchCV(pipeline, param_grid, cv=cv, scoring='neg_root_mean_squared_error', n_jobs=-1)
	model_wbest_params.fit(X, y)
 
	# Therefore, I will now set alpha and l1 ratio to the ones I found
	best_model = model_wbest_params.best_estimator_
	y_pred = best_model.predict(X) # Predict only transforms X_test, not fit_transform/fit X_train (regarding the applyance of StandardScaler, OneHotEncoder, etc... on the Test set) 

	# Get feature names after preprocessing
	onehot_columns = best_model.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_features) # Getting the onehot columns names
	all_columns_names = np.concatenate([numerical_features, onehot_columns]) # Combining the numerical features with the onehot columns names

	# --- Evaluate the model ---
	mae = mean_absolute_error(y, y_pred)
	mape = mean_absolute_percentage_error(y, y_pred)
	r2 = r2_score(y, y_pred)
	rmse = np.sqrt(mean_squared_error(y, y_pred))
 
	print(f'Mean Absolute Error: {mae}') # The average of the absolute differences between predictions and actual values, lower is better
	print(f'Mean Absolute Percentage Error: {mape}') # The average of the absolute percentage differences between predictions and actual values, lower is better
	print(f'R² Score: {r2}') # The proportion of the variance in the dependent variable that is predictable from the independent variable, higher is better
	print(f'Root Mean Squared Error: {rmse}') # The square root of the average of the squared differences between predictions and actual values, lower is better

	# --- Top 5 features ---
	model_coefficients = best_model.named_steps['model'].coef_ # Getting the coefficients of the model
	coef_df = pd.DataFrame({'Features': all_columns_names, 'Coefficient': model_coefficients}) # Creating a dataframe to hold the feature names and their coefficients, for easier sorting

	# Aggregating the coefficients of the one-hot encoded features by their prefix
	coef_df['Prefix'] = coef_df['Features'].apply(lambda x: re.split('_', x, 1)[0] if '_' in x else x)
	aggregated_coef_df = coef_df.groupby('Prefix').agg({'Coefficient': 'sum', 'Features': 'count'}).reset_index()

	# Combine numerical and aggregated categorical features
	numerical_coef_df = coef_df[coef_df['Prefix'].isin(numerical_features)]
	aggregated_coef_df = pd.concat([numerical_coef_df, aggregated_coef_df[~aggregated_coef_df['Prefix'].isin(numerical_features)]])

	# Sort by absolute value of coefficients
	aggregated_coef_df['Abs_Coefficient'] = aggregated_coef_df['Coefficient'].abs()
	aggregated_coef_df = aggregated_coef_df.sort_values(by='Abs_Coefficient', ascending=False)
	aggregated_coef_df['Feature'] = aggregated_coef_df['Prefix']
	top_5_features = aggregated_coef_df.head(5)[['Feature', 'Coefficient']]
	top_5_features['Effect'] = top_5_features['Coefficient'].apply(lambda x: 'Positive' if x > 0 else 'Negative')
	top_5_features.set_index('Feature', inplace=True)
	print(f"\n{top_5_features}")

# Main 

#### Run the line below on your data to see output

In [20]:
prepare_data(df) 

Mean Absolute Error: 7091.318841397638
Mean Absolute Percentage Error: 0.16839546299675717
R² Score: 0.8200940185253495
Root Mean Squared Error: 9430.916494417039

              Coefficient    Effect
Feature                            
model       434799.290523  Positive
manufactor  -57103.484904  Negative
Year         16666.154460  Positive
Prev        -15394.143926  Negative
Gear          7537.271899  Positive


#### End of the code